In [1]:
from pyspark.sql import DataFrame, Row
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import *

from graphframes import *
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import uuid

conf=SparkConf()
# conf.set("spark.executor.memory", "2g")
# conf.set("spark.driver.memory", "12g")
# conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

23/12/26 12:53:22 WARN Utils: Your hostname, office-preacher resolves to a loopback address: 127.0.1.1; using 172.20.10.3 instead (on interface wlp4s0)
23/12/26 12:53:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/26 12:53:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/preacher/anaconda3/envs/delta_run/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
import sys
sys.path.append('/home/preacher/Bungee/CodeRepo/match-management-system/')

from aw_to_mw_synchronizer.transformer.bungee_audit_match_processor import BungeeAuditMatchProcessor
from aw_to_mw_synchronizer.transformer.customer_audit_match_processor import CustomerAuditMatchProcessor
from aw_to_mw_synchronizer.transformer.match_merger import MatchMerger

In [3]:
# audit_library = spark.read.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/customer_audit_library.csv")
# audit_library.show()

In [3]:
args = {
    "customers" : 
        {
            "grocery": ["imperfect<>imperfect", "wholefoods<>wholefoods"],
            "office_supplies" : ["staples<>staples"],
            "pets": ["petco<>petco", "chewy<>chewy"],
            "sports_outdoors": [],
            "petsca" : []
        }
        
}

In [5]:
# audit_match = CustomerAuditMatchProcessor(args, audit_library).process()

In [6]:
# audit_match.show()

In [4]:
fastlane_matches = spark.read.option("header", "true").csv("/home/preacher/Bungee/CodeRepo/match-management-system/data/fastlane.csv")
fastlane_matches.show()

+--------------------+--------------------+-----------+--------------+--------+----------+--------------------+--------------------+----------+--------------------+----------+--------------------+--------------------+----------------+-------------+----+-----+---+-------+--------------------+
|              uuid_a|              uuid_b|   base_upc|      comp_upc|base_sku|  comp_sku|          sku_uuid_a|          sku_uuid_b|     score|   comp_source_store|match_date|              answer|          queue_name|        workflow|inserted_date|year|month|day|segment|   base_source_store|
+--------------------+--------------------+-----------+--------------+--------+----------+--------------------+--------------------+----------+--------------------+----------+--------------------+--------------------+----------------+-------------+----+-----+---+-------+--------------------+
|045125604207<>000...|sku_B06ZYRVV5R<>B...|45125604207|sku_B06ZYRVV5R|     252|B06ZYRVV5R|000252<>petsuppli...|B06ZYRVV5R

In [5]:
bungee_audited_match = BungeeAuditMatchProcessor(args, fastlane_matches).process()

Running bungee_audit_matches


AnalysisException: cannot resolve 'match' given input columns: [aggregated_score, answer, base_sku, base_sku_uuid, base_source_store, base_upc, comp_sku, comp_sku_uuid, comp_source_store, comp_upc, day, inserted_date, match_date, month, pair_id, queue_name, score, segment, sku_uuid_a, sku_uuid_b, uuid_a, uuid_b, workflow, year];
'Project [uuid_a#16, uuid_b#17, base_upc#18, comp_upc#19, base_sku#20, comp_sku#21, sku_uuid_a#22, sku_uuid_b#23, score#24, comp_source_store#178, match_date#26, answer#27, queue_name#28, workflow#29, inserted_date#30, year#31, month#32, day#33, segment#34, base_source_store#157, base_sku_uuid#199, comp_sku_uuid#221, pair_id#244, aggregated_score#268, CASE WHEN Contains('match, exact) THEN EXACT_MATCH WHEN (Contains('match, similar) OR Contains(answer#27, equivalent)) THEN SIMILAR_MATCH ELSE NOT_A_MATCH END AS bungee_audit_status#293]
+- Project [uuid_a#16, uuid_b#17, base_upc#18, comp_upc#19, base_sku#20, comp_sku#21, sku_uuid_a#22, sku_uuid_b#23, score#24, co

In [ ]:
bungee_audited_match.show()

23/12/22 09:38:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+-----------+--------------+--------+----------+--------------------+--------------------+----------+--------------------+----------+--------------------+--------------------+----------------+-------------+----+-----+---+-------+--------------------+--------------------+--------------------+--------------------+----------------+-------------------+-----------------+----------------------+-----------------+--------------------+----------------------+-----------------+--------------------+-------------------------+----------------------+-------------------------+------------+----------+------------+----------+---------+-----------+--------+
|              uuid_a|              uuid_b|   base_upc|      comp_upc|base_sku|  comp_sku|          sku_uuid_a|          sku_uuid_b|     score|   comp_source_store|match_date|              answer|          queue_name|        workflow|inserted_date|year|month|day|segment|   base_source_store|       base_sku_uui